---
title: "Data Gathering R"
---

In [12]:
#| echo: true
#| message: false
#| code-fold: true
#| code-summary: "Libraries"
#| results: 'hide'
#| warning: false

library(tidyverse)
library(ggplot2)
library(forecast)
library(astsa) 
library(xts)
library(tseries)
library(fpp2)
library(fma)
library(lubridate)
library(tidyverse)
library(TSstudio)
library(quantmod)
library(tidyquant)
library(plotly)
library(ggplot2)
library(gridExtra)
library(readxl)
library(zoo)

## Global Lithium Production

In [2]:
df_production <- read.csv("../../data/00-raw-data/lithium-production.csv")

df_production <- df_production %>% filter(nchar(Code) == 3)

head(df_production, n = 10)

#summary(df_production)

,Entity,Code,Year,Lithium.production...kt
,<chr>,<chr>,<int>,<dbl>
1,Argentina,ARG,1995,8
2,Argentina,ARG,1996,8
3,Argentina,ARG,1997,8
4,Argentina,ARG,1998,1130
5,Argentina,ARG,1999,200
6,Argentina,ARG,2000,200
7,Argentina,ARG,2001,200
8,Argentina,ARG,2002,946
9,Argentina,ARG,2003,960


## Chinese Yuan Renminbi to U.S. Dollar Spot Exchange Rate

In [3]:
#| echo: true
#| message: false
#| code-fold: true
#| code-summary: "Libraries"
#| warning: false

# Set the start and end dates
start_date <- "2010-01-01"
end_date <- "2022-12-31"

# Define the symbol for CNY to USD exchange rate
symbol <- "DEXCHUS"

# Use getSymbols() to fetch the data
getSymbols(symbol, from = start_date, to = end_date, src = "FRED")

# Access the data as a data frame
df_exchange_rate <- as.data.frame(DEXCHUS)

df_exchange_rate <- rownames_to_column(df_exchange_rate, var = "DATE")

df_exchange_rate$DATE <- as.Date(df_exchange_rate$DATE)

# Print the first few rows of the data
head(df_exchange_rate)

summary(df_exchange_rate)

[1] "DEXCHUS"

,DATE,DEXCHUS
,<date>,<dbl>
1,2010-01-01,NA
2,2010-01-04,6.8273
3,2010-01-05,6.8258
4,2010-01-06,6.8272
5,2010-01-07,6.8280
6,2010-01-08,6.8274


      DATE               DEXCHUS     
 Min.   :2010-01-01   Min.   :6.040  
 1st Qu.:2013-04-02   1st Qu.:6.309  
 Median :2016-07-01   Median :6.504  
 Mean   :2016-07-01   Mean   :6.548  
 3rd Qu.:2019-10-01   3rd Qu.:6.805  
 Max.   :2022-12-30   Max.   :7.305  
                      NA's   :140    

## Commodity Price

### Uranium

In [ ]:
#| echo: true
#| message: false
#| code-fold: true
#| code-summary: "Data Cleaning Code"
#| results: 'hide'
#| warning: false

df_commodity_price <- read_excel("../../data/00-raw-data/commodity_price.xlsx")

df_commodity_price <- df_commodity_price %>%
  pivot_longer(cols = -c('...1'), 
               names_to = "Month_Year",
               values_to = "Price")

df_commodity_price <- df_commodity_price %>% filter(!is.na(Price) & Price != "")

df_commodity_price$Month_Year <- as.yearmon(df_commodity_price$Month_Year, format = "%b %Y")

df_commodity_price$Month_Year <- format(df_commodity_price$Month_Year, "%m-%Y")

df_commodity_price$Month_Year <- paste("01-", df_commodity_price$Month_Year, sep = "")

df_commodity_price$Month_Year <- as.Date(df_commodity_price$Month_Year, format = "%d-%m-%Y")

names(df_commodity_price) <- c('Commodity', 'DATE', 'Price')

df_uranium_price <- df_commodity_price %>%
  filter(Commodity == "Uranium")

#df_uranium_price <- head(df_uranium_price, n = 10)


In [11]:
head(df_uranium_price,20)

Commodity,DATE,Price
<chr>,<date>,<dbl>
Uranium,2012-01-01,52.31250
Uranium,2012-02-01,52.05556
Uranium,2012-03-01,51.28889
Uranium,2012-04-01,51.30000
Uranium,2012-05-01,51.88889
Uranium,2012-06-01,50.83333
Uranium,2012-07-01,50.35556
Uranium,2012-08-01,49.25000
Uranium,2012-09-01,47.72500


### Natural Gas

In [7]:
# Set the start and end dates
start_date <- "2012-01-01"
end_date <- "2022-12-31"

# Define the symbol for Gas Price to USD exchange rate
symbol <- "GASREGCOVW"

# Use getSymbols() to fetch the data
getSymbols(symbol, from = start_date, to = end_date, src = "FRED")

# Access the data as a data frame
df_gas_price <- as.data.frame(GASREGCOVW)

df_gas_price <- rownames_to_column(df_gas_price, var = "DATE")

df_gas_price$DATE <- as.Date(df_gas_price$DATE)

# Print the first few rows of the data
head(df_gas_price, 10)

[1] "GASREGCOVW"

,DATE,GASREGCOVW
,<date>,<dbl>
1,2012-01-02,3.254
2,2012-01-09,3.333
3,2012-01-16,3.342
4,2012-01-23,3.333
5,2012-01-30,3.386
6,2012-02-06,3.436
7,2012-02-13,3.466
8,2012-02-20,3.523
9,2012-02-27,3.641


In [8]:
df_gas_price <- df_gas_price %>%
    mutate(Year = year(DATE)) %>%
    mutate(Month = month(DATE)) %>%
    mutate(Day = day(DATE))

df_gas_price <- df_gas_price %>%
    group_by(Year, Month) %>%
    filter(Day == min(Day)) %>%
    ungroup()

df_gas_price <- df_gas_price %>%
    select(GASREGCOVW, Year, Month, Day)

df_gas_price <- df_gas_price %>%
    mutate(DATE = paste(Year, Month, '01', sep = "-"))

df_gas_price$DATE <- as.Date(df_gas_price$DATE)

df_gas_price <- df_gas_price %>%
    select(GASREGCOVW, DATE)

head(df_gas_price, 10)

GASREGCOVW,DATE
<dbl>,<date>
3.254,2012-01-01
3.436,2012-02-01
3.717,2012-03-01
3.874,2012-04-01
3.718,2012-05-01
3.518,2012-06-01
3.291,2012-07-01
3.606,2012-08-01
3.797,2012-09-01


### Resources Prices

In [9]:
df_uranium_price <- df_uranium_price %>% select(DATE, Price)

names(df_uranium_price) <- c('DATE', 'Uranium')

names(df_gas_price) <- c('Natural Gas', 'DATE')

df_resource_price <- merge(df_uranium_price, df_gas_price, by.x = 'DATE', by.y = 'DATE', all = TRUE)

df_resource_price <- na.omit(df_resource_price)

# head(df_resource_price, 10)

df_resource_price

,DATE,Uranium,Natural Gas
,<date>,<dbl>,<dbl>
1,2012-01-01,52.31250,3.254
2,2012-02-01,52.05556,3.436
3,2012-03-01,51.28889,3.717
4,2012-04-01,51.30000,3.874
5,2012-05-01,51.88889,3.718
6,2012-06-01,50.83333,3.518
7,2012-07-01,50.35556,3.291
8,2012-08-01,49.25000,3.606
9,2012-09-01,47.72500,3.797
